# In this notebook we are going to cover the most fundamental concepts of tensors using tensorflow

## More specfifically, we're going to cover:
    1) Introduction to tensors
    2) Getting information from tensors
    3) manipulating tensors
    4) Tensors & NumPy
    5) using @tf.function (a way to speed up your python functions)
    6) Using GPUs with TensorFlow (or TPUs)
    7) Exercises to try yourself!

## Introduction to Tensors

In [6]:
# Import TensorFLow
import tensorflow as tf 
print(tf.__version__)

2.10.1


In [5]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
print(scalar)

tf.Tensor(7, shape=(), dtype=int32)


2022-11-28 16:11:29.975672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
# Check the number of dimensions of a tensor (ndim stands for number of dimentions)
scalar.ndim

0

In [29]:
# Create a vector 
vector = tf.constant([10,10])
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [32]:
# Check the dimention of our vector
print(vector.ndim)

1


In [34]:
# Create a matrix (has more than 1 dimention)
matrix = tf.constant([[10,7],
                      [7, 10]])
print(matrix)

tf.Tensor(
[[10  7]
 [ 7 10]], shape=(2, 2), dtype=int32)


In [36]:
# Check dimention of matrix
print(matrix.ndim)

2


In [38]:
# Create another matrix with shorter memory
another_matrix = tf.constant([[10., 7.], 
                              [7., 10.], 
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
print(another_matrix) # notice dtype

tf.Tensor(
[[10.  7.]
 [ 7. 10.]
 [ 8.  9.]], shape=(3, 2), dtype=float16)


In [40]:
# Number of dimention of "another_matrix"
print(another_matrix.ndim)

2


In [7]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9], 
                       [10, 11, 12]], 
                      [[13, 14, 15],
                       [16, 17, 18]]])
print(tensor)

tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]], shape=(3, 2, 3), dtype=int32)


In [42]:
# Check the dimentions of the tensor
print(tensor.ndim)

3


# What we've created so far:
* Scalar: a signle number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a two-dimentional array of numbers
* Tensor: an n-dimentional array of numbers (where n can be any number, a 0-dimentional tensor is a scalar, a 1-dimentional tensor is a vector)

### Creating tensors with `tf.Variable`

In [13]:
# Create the same tensor as above but with tf.Variable()
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
print(changeable_tensor, unchangeable_tensor)

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)> tf.Tensor([10  7], shape=(2,), dtype=int32)


In [14]:
# Let's try to change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [16]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [54]:
# Let's try to change our unchangeable_tensor
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

**Note:** Rarely in practice will you need to decide whether the use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating Random Tensors
Random tensors are tensors of some arbitrary size with random numbers.

In [21]:
# Create two random (but the same) tensors
# "seed" allows tensorflow to geneate random numbers using a ~flavor~, "x". Where "x" is the seed.
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducability
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Suffle the order of elements in a tensor
This is useful for situations like image classifiication of ramen or spaghetti. Say you have 10k images and the first 8k images are of ramen. Your classifier will become biased to ramen because your dataset has not been randomized.

In [38]:
# Shuffle a tensor (valuable for when want to shuffle your data so the inherent order doesnt affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

In [53]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42) # global level seed
tf.random.shuffle(not_shuffled, seed=42) # operational level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**Exersize:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.